In [1]:
import re
from unidecode import unidecode
import pandas as pd
from ibge.localidades import Municipios

cidades = Municipios().json()
cidades = {cidade['nome']: cidade['microrregiao']['mesorregiao']['UF']['sigla'] for cidade in cidades}

linkedin = pd.read_excel('dados/vagas.xlsx', sheet_name='LinkedIn')
vagas = pd.read_excel('dados/vagas.xlsx', sheet_name='vagas.com.br')

In [2]:
print(linkedin.info())
print(vagas.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851 entries, 0 to 850
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Data         851 non-null    object
 1   Empresa      851 non-null    object
 2   Título       851 non-null    object
 3   Localização  851 non-null    object
 4   Descrição    851 non-null    object
 5   Nível        851 non-null    object
 6   Tipo         851 non-null    object
 7   Função       851 non-null    object
 8   Indústria    851 non-null    object
 9   Link         851 non-null    object
dtypes: object(10)
memory usage: 66.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722 entries, 0 to 721
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Data         722 non-null    object
 1   Título       722 non-null    object
 2   Empresa      722 non-null    object
 3   Descrição    722 non-null    object
 4   L

In [3]:
linkedin['Data'] = pd.to_datetime(linkedin['Data'], errors='coerce', format='%Y-%m-%d')
vagas['Data'] = pd.to_datetime(vagas['Data'], errors='coerce', format='%d/%m/%Y')

In [4]:
cols = ['Data', 'Título', 'Empresa', 'Descrição', 'Localização', 'Nível', 'Link']
linkedin = linkedin.loc[:, cols]
vagas = vagas.loc[:, cols]

df = pd.concat((linkedin, vagas), axis=0)
df.shape

(1573, 7)

In [5]:
df['Nível'] = df['Nível'].str.replace('[/\n\s]+', '/', regex=True)
df['Nível'].unique()

array(['Não/aplicável', 'Pleno-sênior', 'Júnior', 'Assistente', 'Estágio',
       'Executivo', 'Pleno', 'Sênior', 'Júnior/Trainee',
       'Auxiliar/Operacional', 'Supervisão/Coordenação', 'Técnico',
       'Gerência'], dtype=object)

In [6]:
def identifica_cargo(titulo):
    kwmap = {
        'Cientista de Dados': [('cientista', 'dados'), ('data', 'scientist'), ('data', 'science'), ('ciencia', 'dados')],
        'Analista de Dados': [('analista', 'dados'), ('data', 'analyst'), ('analise', 'dados'), ('data', 'analytics')],
        'Analista de Negócios': [('analista', 'negocio'), ('business', 'analyst'), ('business', 'analytics')],
        'Engenheiro(a) de Dados': [('engenheiro', 'dados'), ('data', 'engineer'), ('engenharia', 'dados'), ('engenheira', 'dados')],
        'Assistente de Dados': [('assistente', 'dados')],
        'Auxiliar de Dados': [('auxiliar', 'dados')],
        'Analista de BI': [('analista', 'bi'), ('bi', 'analyst')],
        'Especialista de Dados': [('especialista', 'dados'), ('data', 'expert')],
        'Estagiário em BI': [('estagiario', 'bi'), ('estagiario', 'business', 'intelligence')]
    }
    
    words = unidecode(titulo).lower().split(' ')
    
    for cargo, kws in kwmap.items():
        for kw in kws:
            exists = [True if w in words else False for w in kw]
            if all(exists):
                return cargo
            
    return pd.NA

df['Cargo'] = df['Título'].apply(identifica_cargo)
df['Cargo'].value_counts(sort=True, ascending=False, dropna=False)

<NA>                      1176
Analista de Dados          248
Cientista de Dados          64
Engenheiro(a) de Dados      32
Analista de BI              19
Especialista de Dados       16
Assistente de Dados          7
Estagiário em BI             5
Analista de Negócios         4
Auxiliar de Dados            2
Name: Cargo, dtype: int64

In [7]:
df.dropna(subset=['Cargo'], inplace=True)
df.shape

(397, 8)

In [8]:
df['Localização'].unique()

array(['São Paulo, SP', 'Florianópolis, SC', 'Nova Iguaçu, RJ',
       'Curitiba, PR', 'Recife, PE', 'Rio de Janeiro, RJ', 'Brasil',
       'Porto Alegre e Região', 'Belo Horizonte, MG', 'Agudos, SP',
       'Vinhedo, SP', 'Paracatu, MG', 'Santa Catarina, Brasil',
       'Brasília, DF', 'Barueri, SP', 'Recife e Região', 'Jandira, SP',
       'Valinhos, SP', 'São Paulo, Brasil', 'Sorocaba, SP',
       'Nova Lima, MG', 'Rio Verde, GO', 'São José dos Campos, SP',
       'Porto Alegre, RS', 'Contagem, MG', 'Guarulhos, SP',
       'Piracicaba, SP', 'Goiânia, GO', 'Marechal Deodoro, AL',
       'Vitória, ES', 'Rio do Sul, SC', 'Ponta Grossa, PR',
       'Juiz de Fora, MG', 'Monte Mor, SP', 'Florianópolis e Região',
       'Macaé, RJ', 'Osasco, SP', 'Campinas e Região', 'Cascavel, PR',
       'Palhoça, SC', 'São Paulo e Região', 'Campinas, SP',
       'São José, SC', 'Farroupilha, RS', 'Aracruz, ES',
       'São José do Rio Preto, SP', 'Mogi das Cruzes, SP',
       'Fortaleza, CE', 'Belém, PA

In [9]:
def processa_localizacao(localizacao):
    regex = '^(.+)(, | e )(.+)$'
    s = re.search(regex, localizacao)
    if s:
        cidade = s.group(1)
        if cidade in cidades:
            return f'{cidade}/{cidades[cidade]}'

    return localizacao.replace(', ','/')
    
df['Localização'] = df['Localização'].apply(processa_localizacao)

In [10]:
df = df.reset_index().drop('index', axis=1)
with pd.ExcelWriter('dados/vagas_processado.xlsx', engine='auto') as writer:
    df.to_excel(writer, 'Vagas', index=False)